# GPU acceleration

In [1]:
import time
from functools import wraps

import numpy as np

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Function '{func.__name__}' executed in {end - start:.4f} seconds")
        return result

    return wrapper

In [3]:
try:
    import google.colab

    IN_COLAB = True
    !pip install git+https://github.com/mark-hobbs/pypd.git@feature/gpu
    print("Package installed successfully")
    import pypd
except ImportError:
    IN_COLAB = False
    import pypd

    print("Not running in Colab")

Not running in Colab


In [4]:
if IN_COLAB:
    try:
        gpu_info = !nvidia-smi
        gpu_info = "\n".join(gpu_info)
        print("GPU Information:")
        print(gpu_info)
    except:
        print("GPU information not available")

    try:
        import multiprocessing

        cpu_info = f"Number of CPU cores: {multiprocessing.cpu_count()}"
        print("\nCPU Information:")
        print(cpu_info)
    except:
        print("CPU information not available")
else:
    print("Not running in Colab.")

Not running in Colab.


## Build model

Crack branching

In [5]:
def build_particle_coordinates(dx, n_div_x, n_div_y):
    particle_coordinates = np.zeros([n_div_x * n_div_y, 2])
    counter = 0

    for i_y in range(n_div_y):  # depth
        for i_x in range(n_div_x):  # length
            coord_x = dx * i_x
            coord_y = dx * i_y
            particle_coordinates[counter, 0] = coord_x
            particle_coordinates[counter, 1] = coord_y
            counter += 1

    return particle_coordinates

In [6]:
def build_boundary_conditions(particles, dx):
    bc_flag = np.zeros((len(particles), 2), dtype=np.intc)
    bc_unit_vector = np.zeros((len(particles), 2), dtype=np.intc)

    tol = 1e-6

    for i, particle in enumerate(particles):
        if particle[1] < (0.02 + tol):
            bc_flag[i, 1] = 1
            bc_unit_vector[i, 1] = -1
        if particle[1] > (0.18 - dx - tol):
            bc_flag[i, 1] = 1
            bc_unit_vector[i, 1] = 1

    return bc_flag, bc_unit_vector

In [7]:
dx = 1e-3
n_div_x = np.rint(0.4 / dx).astype(int)
n_div_y = np.rint(0.2 / dx).astype(int)
notch = [np.array([0 - dx, 0.1 - (dx / 2)]), np.array([0.2, 0.1 - (dx / 2)])]

In [8]:
x = build_particle_coordinates(dx, n_div_x, n_div_y)
flag, unit_vector = build_boundary_conditions(x, dx)

In [9]:
material = pypd.Material(name="homalite", E=4.55e9, Gf=38.46, density=1230, ft=2.5)
bc = pypd.BoundaryConditions(flag, unit_vector, magnitude=1e-4)
particles = pypd.ParticleSet(x, dx, bc, material)
bonds = pypd.BondSet(particles, influence=pypd.Constant, notch=notch)
model = pypd.Model(particles, bonds)

/Users/markhobbs/Documents/02-repositories/pypd/pypd/tools.py:93: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = alpha_numerator / denominator
/Users/markhobbs/Documents/02-repositories/pypd/pypd/tools.py:94: RuntimeWarning: divide by zero encountered in scalar divide
  beta = beta_numerator / denominator


In [10]:
simulation = pypd.Simulation(n_time_steps=5000, damping=0)

Is CUDA available: False


## Speed testing: particle forces

In [11]:
@timeit
def compute_nodal_forces_a(
    x,
    u,
    cell_volume,
    bondlist,
    d,
    c,
    f_x,
    f_y,
    material_law,
    surface_correction_factors,
):
    """
    Compute particle forces - employs bondlist

    Parameters
    ----------
    bondlist : ndarray (int)
        Array of pairwise interactions (bond list)

    x : ndarray (float)
        Material point coordinates in the reference configuration

    u : ndarray (float)
        Nodal displacement

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed

    c : float
        Bond stiffness

    material_law : function

    Returns
    -------
    node_force : ndarray (float)
        Nodal force array

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed
    """

    n_nodes = np.shape(x)[0]
    n_dimensions = np.shape(x)[1]
    n_bonds = np.shape(bondlist)[0]
    node_force = np.zeros((n_nodes, n_dimensions))

    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        xi_x = x[node_j, 0] - x[node_i, 0]
        xi_y = x[node_j, 1] - x[node_i, 1]

        xi_eta_x = xi_x + (u[node_j, 0] - u[node_i, 0])
        xi_eta_y = xi_y + (u[node_j, 1] - u[node_i, 1])

        xi = np.sqrt(xi_x**2 + xi_y**2)
        y = np.sqrt(xi_eta_x**2 + xi_eta_y**2)
        stretch = (y - xi) / xi

        d[k_bond] = material_law(k_bond, stretch, d[k_bond])

        f = (
            stretch
            * c[k_bond]
            * (1 - d[k_bond])
            * cell_volume
            * surface_correction_factors[k_bond]
        )
        f_x[k_bond] = f * xi_eta_x / y
        f_y[k_bond] = f * xi_eta_y / y

    # Reduce bond forces to particle forces
    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        node_force[node_i, 0] += f_x[k_bond]
        node_force[node_j, 0] -= f_x[k_bond]
        node_force[node_i, 1] += f_y[k_bond]
        node_force[node_j, 1] -= f_y[k_bond]

    return node_force, d

In [12]:
from numba import njit, prange

@timeit
@njit(parallel=True, fastmath=True)
def compute_nodal_forces_b(
    x,
    u,
    cell_volume,
    bondlist,
    d,
    c,
    f_x,
    f_y,
    material_law,
    surface_correction_factors,
):
    """
    Compute particle forces - employs bondlist

    Parameters
    ----------
    bondlist : ndarray (int)
        Array of pairwise interactions (bond list)

    x : ndarray (float)
        Material point coordinates in the reference configuration

    u : ndarray (float)
        Nodal displacement

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed

    c : float
        Bond stiffness

    material_law : function

    Returns
    -------
    node_force : ndarray (float)
        Nodal force array

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed
    """

    n_nodes = np.shape(x)[0]
    n_dimensions = np.shape(x)[1]
    n_bonds = np.shape(bondlist)[0]
    node_force = np.zeros((n_nodes, n_dimensions))

    for k_bond in prange(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        xi_x = x[node_j, 0] - x[node_i, 0]
        xi_y = x[node_j, 1] - x[node_i, 1]

        xi_eta_x = xi_x + (u[node_j, 0] - u[node_i, 0])
        xi_eta_y = xi_y + (u[node_j, 1] - u[node_i, 1])

        xi = np.sqrt(xi_x**2 + xi_y**2)
        y = np.sqrt(xi_eta_x**2 + xi_eta_y**2)
        stretch = (y - xi) / xi

        d[k_bond] = material_law(k_bond, stretch, d[k_bond])

        f = (
            stretch
            * c[k_bond]
            * (1 - d[k_bond])
            * cell_volume
            * surface_correction_factors[k_bond]
        )
        f_x[k_bond] = f * xi_eta_x / y
        f_y[k_bond] = f * xi_eta_y / y

    # Reduce bond forces to particle forces
    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        node_force[node_i, 0] += f_x[k_bond]
        node_force[node_j, 0] -= f_x[k_bond]
        node_force[node_i, 1] += f_y[k_bond]
        node_force[node_j, 1] -= f_y[k_bond]

    return node_force, d

In [13]:
compute_nodal_forces_a(particles.x, 
                       particles.u, 
                       particles.cell_volume,
                       bonds.bondlist, 
                       bonds.d, 
                       bonds.c, 
                       bonds.f_x,
                       bonds.f_y,
                       bonds.constitutive_law.calculate_bond_damage, 
                       bonds.surface_correction_factors);

compute_nodal_forces_b(particles.x, 
                       particles.u, 
                       particles.cell_volume,
                       bonds.bondlist, 
                       bonds.d, 
                       bonds.c, 
                       bonds.f_x,
                       bonds.f_y,
                       bonds.constitutive_law.calculate_bond_damage, 
                       bonds.surface_correction_factors);

Function 'compute_nodal_forces_a' executed in 3.6575 seconds
Function 'compute_nodal_forces_b' executed in 0.4370 seconds


## Speed testing: update particle positions 

In [14]:
import numpy as np

@timeit
def euler_cromer_a(
    node_force,
    u,
    v,
    a,
    damping,
    node_density,
    dt,
    bc_flag,
    bc_magnitude,
    bc_unit_vector,
):
    """
    Update particle positions using an Euler-Cromer time integration scheme

    Parameters
    ----------
    u : ndarray (float)
        Particle displacement

    v : ndarray (float)
        Particle velocity

    a : ndarray (float)
        Particle acceleration
    """

    n_nodes = np.shape(node_force)[0]
    n_dimensions = np.shape(node_force)[1]

    for node_i in range(n_nodes):
        for dof in range(n_dimensions):
            a[node_i, dof] = (
                node_force[node_i, dof] - damping * v[node_i, dof]
            ) / node_density
            v[node_i, dof] = v[node_i, dof] + (a[node_i, dof] * dt)
            u[node_i, dof] = u[node_i, dof] + (v[node_i, dof] * dt)

            if bc_flag[node_i, dof] != 0:
                u[node_i, dof] = bc_magnitude * bc_unit_vector[node_i, dof]

    return u, v

In [15]:
import numpy as np
from numba import njit, prange

@timeit
@njit(parallel=True)
def euler_cromer_b(
    node_force,
    u,
    v,
    a,
    damping,
    node_density,
    dt,
    bc_flag,
    bc_magnitude,
    bc_unit_vector,
):
    """
    Update particle positions using an Euler-Cromer time integration scheme

    Parameters
    ----------
    u : ndarray (float)
        Particle displacement

    v : ndarray (float)
        Particle velocity

    a : ndarray (float)
        Particle acceleration
    """

    n_nodes = np.shape(node_force)[0]
    n_dimensions = np.shape(node_force)[1]

    for node_i in prange(n_nodes):
        for dof in range(n_dimensions):
            a[node_i, dof] = (
                node_force[node_i, dof] - damping * v[node_i, dof]
            ) / node_density
            v[node_i, dof] = v[node_i, dof] + (a[node_i, dof] * dt)
            u[node_i, dof] = u[node_i, dof] + (v[node_i, dof] * dt)

            if bc_flag[node_i, dof] != 0:
                u[node_i, dof] = bc_magnitude * bc_unit_vector[node_i, dof]

    return u, v

In [16]:
euler_cromer_a(
    particles.f,
    particles.u,
    particles.v,
    particles.a,
    simulation.damping,
    particles.node_density,
    1,
    particles.bc.flag,
    1,
    particles.bc.unit_vector,
)

euler_cromer_b(
    particles.f,
    particles.u,
    particles.v,
    particles.a,
    simulation.damping,
    particles.node_density,
    1,
    particles.bc.flag,
    1,
    particles.bc.unit_vector,
)

Function 'euler_cromer_a' executed in 0.2376 seconds
Function 'euler_cromer_b' executed in 0.2110 seconds


(array([[ 0., -1.],
        [ 0., -1.],
        [ 0., -1.],
        ...,
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]))

## Numba CUDA

In [17]:
model._allocate_gpu_arrays()

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: